<a href="https://colab.research.google.com/github/Ayumi51482/rally-flask-app/blob/main/rally_kf73.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
## import and install
!pip install flask pandas gspread matplotlib pytz pyngrok

from oauth2client.service_account import ServiceAccountCredentials
from collections import defaultdict
from datetime import datetime
import subprocess
import sys
import pytz
from flask import Flask, render_template, request, redirect, url_for
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pyngrok import ngrok
# ここに取得したAuthtokenを入力
ngrok.set_auth_token('2rBuL3S8ZEZDcNiM7NqWRXBPL7I_3wwAEYaxH6dmuGUXtryqD')
from collections import defaultdict
import os
import shutil
import threading
import datetime

In [ ]:
# 認証情報のファイルパス、公開鍵のファイルを手動でアップロードするべし
creds = ServiceAccountCredentials.from_json_keyfile_name(
    'rally-kf73-21a154fc2a7e.json',
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
)
# 認証してGoogle Sheets APIにアクセス
client = gspread.authorize(creds)


In [ ]:
# 日本時間(JST)を指定
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
current_time_jst = datetime.datetime.now(jst).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
##ここからFlaskでアプリを実装していく

In [ ]:
# Sheet1の参照（シート名が「Sheet1」だと仮定）
sheet = client.open('rally-kf73').worksheet('生データ')
# Sheet2の参照（シート名が「Sheet2」だと仮定）
sheet2 = client.open('rally-kf73').worksheet('時間ごと集計')

# ヘッダーを設定
header1 = ['日付', '時間', 'AAA', 'BBB', 'CCC', '年代']
sheet.insert_row(header1 ,1)
header2 = ['日付', '時間', 'AAAのクリア人数', 'BBBのクリア人数', 'CCCのクリア人数', '合計人数']
sheet2.insert_row(header2 ,1)

records = sheet.get_all_records()
print(len(records))  # データが何件取得されたか確認
if len(records) > 0:
    print(records[0])  # 最初のレコードを表示してカラム名を確認
else:
    print("シートにデータがありません。")

1
{'日付': '日付', '時間': '時間', 'AAA': 'AAA', 'BBB': 'BBB', 'CCC': 'CCC', '年代': '年代'}


In [ ]:
###実行する時はここ###


# Flaskアプリの作成
app = Flask(__name__)

# 認証情報のセットアップ
creds = ServiceAccountCredentials.from_json_keyfile_name(
    'rally-kf73-21a154fc2a7e.json',
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
)
client = gspread.authorize(creds)

## 送信画面のルーティング
@app.route('/')
def index():
     return render_template('index.html', current_time=current_time_jst)

##フォーム送信のルーティング
@app.route('/submit', methods=['POST'])
def submit():
    # 現在時刻を取得
    jst = pytz.timezone('Asia/Tokyo')
    current_day_jst = datetime.datetime.now(jst).strftime('%Y-%m-%d')
    current_time_jst = datetime.datetime.now(jst).strftime('%H:%M:%S')

    # フォームデータを取得
    selected_items = request.form.getlist('clear_count')  # 選択されたチェックボックスのリスト
    age_group = request.form.get('age_group')  # 年代

    # AAA, BBB, CCC の選択状態を確認して 1 or 0 に変換
    aaa = 1 if 'AAA' in selected_items else 0
    bbb = 1 if 'BBB' in selected_items else 0
    ccc = 1 if 'CCC' in selected_items else 0

    # スプレッドシートにデータを記録
    sheet.append_row([current_day_jst, current_time_jst, aaa, bbb, ccc, age_group])

    return redirect(url_for('index'))

## Flaskアプリ起動
if __name__ == '__main__':
    app.run(debug=False)  # HerokuやRenderではデバッグモードを無効化

######Cloudで固定URLにするため使わない
# # ngrokのトンネルを作成
# public_url = ngrok.connect(5000)
# print('Flask app is publicly accessible at:', public_url)

# # Flaskアプリをバックグラウンドで実行するためのスレッドを作成
# def run_flask():
#     app.run(debug=True, port=5000, use_reloader=False, use_debugger=False)
# if __name__ == '__main__':
#     threading.Thread(target=run_flask).start()  # Flaskアプリをバックグラウンドで実行

In [8]:
##herokuのログインがうまくいかないので、renderでやるよん
# ライブラリをインストールする関数
def install_package(package_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

# 必要なライブラリをリストアップ
packages = ["gitpython"]

# ライブラリを順にインストール
for package in packages:
    try:
        install_package(package)
        print(f"Successfully installed {package}")
    except Exception as e:
        print(f"Error installing {package}: {e}")

In [9]:
import os
from git import Repo

# 1. リポジトリの情報を設定
LOCAL_REPO_PATH = "./my_flask_app"  # プロジェクトのディレクトリ
REMOTE_URL = "https://github.com/<your-username>/<repository-name>.git"  # GitHubリポジトリURL

# 2. ローカルリポジトリを初期化
if not os.path.exists(LOCAL_REPO_PATH):
    os.makedirs(LOCAL_REPO_PATH)
    print(f"Created directory: {LOCAL_REPO_PATH}")

repo = Repo.init(LOCAL_REPO_PATH)
print(f"Initialized Git repository at: {LOCAL_REPO_PATH}")

# 3. ファイルを追加
# 必要に応じてプロジェクトファイルを準備する
with open(os.path.join(LOCAL_REPO_PATH, "app.py"), "w") as f:
    f.write("from flask import Flask\napp = Flask(__name__)\n\n@app.route('/')\ndef hello():\n    return 'Hello, World!'\n\nif __name__ == '__main__':\n    app.run(debug=True)")

with open(os.path.join(LOCAL_REPO_PATH, "requirements.txt"), "w") as f:
    f.write("flask\n")

repo.index.add(["app.py", "requirements.txt"])
repo.index.commit("Initial commit with Flask app")

# 4. リモートリポジトリを設定
origin = repo.create_remote("origin", REMOTE_URL)
print(f"Added remote repository: {REMOTE_URL}")

# 5. プッシュ操作
try:
    repo.git.push("--set-upstream", "origin", "main")
    print(f"Pushed to {REMOTE_URL}")
except Exception as e:
    print(f"Error pushing to remote: {e}")

Created directory: ./my_flask_app
Initialized Git repository at: ./my_flask_app
Added remote repository: https://github.com/<your-username>/<repository-name>.git
Error pushing to remote: Cmd('git') failed due to: exit code(1)
  cmdline: git push --set-upstream origin main
  stderr: 'error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/<your-username>/<repository-name>.git''


In [ ]:
with open("index.html", "w", encoding="utf-8") as f:
    f.write("""<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>スタンプラリー受付</title>
</head>
<body>
    <h1>スタンプラリー受付</h1>
    <p>現在の時刻: {{ current_time }}</p>

    <form method="POST" action="/submit">
    <h3>クリア数を選択してください:</h3>
    <input type="checkbox" id="AAA" name="clear_count" value="AAA"> AAA
    <input type="checkbox" id="BBB" name="clear_count" value="BBB"> BBB
    <input type="checkbox" id="CCC" name="clear_count" value="CCC"> CCC
    <br><br>

    <label for="age_group">年代:</label><br>
    <select name="age_group" id="age_group">
        <option value="中高生">中高生</option>
        <option value="大学生-20代">大学生-20代</option>
        <option value="30代-60代">30代-60代</option>
        <option value="60代以上">60代以上</option>
    </select>
    <br><br>

    <button type="submit">送信</button>
    </form>
</body>
</html>
  """)

In [ ]:
# 'index.html'ファイルを作成して保存
# templatesというフォルダを作成
os.makedirs('templates', exist_ok=True)


## index.htmlをtemplateフォルダに移籍
source = "index.html"
destination_folder = "templates"
shutil.move(source, os.path.join(destination_folder, source))
print(f"'{source}' を '{destination_folder}' フォルダに移動しました！")

'index.html' を 'templates' フォルダに移動しました！


In [ ]:
## ngrokが３個以上働いているから実行できない時の対処法

subprocess.check_call([sys.executable, "-m", "pip", "install", "psutil"])
import psutil

# ngrokのプロセスを全て終了させる関数
def kill_ngrok():
    for proc in psutil.process_iter(['pid', 'name']):
        if 'ngrok' in proc.info['name'].lower():
            proc.kill()
            print(f"Killed ngrok process: {proc.info['pid']}")

# ngrokを終了させる
kill_ngrok()

# 'ngrok'プロセスが存在するか確認
def check_ngrok_running():
    for proc in psutil.process_iter(['pid', 'name']):
        if 'ngrok' in proc.info['name'].lower():
            return True
    return False

# ngrokが実行中かどうかを確認
if check_ngrok_running():
    print("ngrok is running")
else:
    print("ngrok is not running")


ngrok is not running
